## Event reproducibility in RSC
### data from Franco and Goard 2021
See the paper:    
*"A distributed circuit for associating environmental context with motor choice in retrosplenial cortex"*    
Luis M. Franco Michael J. Goard    
Science Advances 7, eabf9815 (2021). doi:10.1126/sciadv.abf9815

In [1]:
from platform import python_version
print(python_version())

%run -i 'imports_functions.py' 

%matplotlib inline

3.10.4


### Data preprocessing

The original file `twoPhotonData.mat` containing the DF/F traces is available on [Zenodo](https://zenodo.org/record/4734407/files/twoPhotonData.mat).

Here, to speed up the processing and reduce the dependencies, we use the Ca spiketrains extracted using the `deconvolve` function available in the library [OASIS](https://github.com/j-friedrich/OASIS/pull/19) (with all default parameters, and `penalty` of 1).

In [2]:
if os.path.exists('goard/spiketrains_twoPhotonData_unselected.npy'):
    all_spiketrains = np.load('goard/spiketrains_twoPhotonData_unselected.npy', allow_pickle=True)
    time = np.load('goard/time_twoPhotonData_unselected.npy', allow_pickle=True) # all time samples
    print("... loaded",len(all_spiketrains), "spiketrains for", len(time), "intervals")

exp_path = os.getcwd() + "/goard/"

frame_duration = 0.1 # sec, 10 frames per second, see Methods in Franco and Goard 2021

... loaded 7770 spiketrains for 20160 intervals


We separate the 29 sessions using the `cellsPerSession` field of the matlab file (here listed for simplicity).

In [3]:
cellsPerSession = [
    279, 136, 317, 296, 292, 281, 258, 279, 279, 271, 
    177, 257, 296, 253, 329, 230, 292, 181, 201, 292, 
    241, 382, 239, 249, 315, 262, 237, 321, 328
]
indices_cellsPerSession = np.cumsum(cellsPerSession)
print(indices_cellsPerSession)

sessions = np.split(all_spiketrains, indices_cellsPerSession)[:-1] # omit last empty split


[ 279  415  732 1028 1320 1601 1859 2138 2417 2688 2865 3122 3418 3671
 4000 4230 4522 4703 4904 5196 5437 5819 6058 6307 6622 6884 7121 7442
 7770]


## Dynamical analysis

Analysis for each session.

On average, the longest responses were for the 4th trial. We take it.

In [ ]:
global_events = [[] for i in range(29)] # by session
global_events_vector = [[] for i in range(29)] # by session
global_events_sec = []
global_events_duration = []
global_cluster_number = []
global_cluster_selfsimilarity = [[] for i in range(29)]

core_reproducibility_perc = 99.9 # threshold for detecting cores

# there are session with NaNs
sessions_evt_toBremoved = [[] for i in range(29)] # by session
sessions_evt_toBremoved[0] = [22] 
sessions_evt_toBremoved[2] = [30]
sessions_evt_toBremoved[10] = [60,61]
sessions_evt_toBremoved[17] = [63,64,65,71,86,87,88]

all_pagerank_cores = []
all_pagerank_others = []

for scan_id,scan_spiketrains in enumerate(sessions):
    print("\n\n%d/%d" % (scan_id+1,len(sessions)))
    print("... loaded %s spiketrains"%(len(scan_spiketrains)))
    
    # we only take trial 4, starting at 1500 sec
    scan_spiketrains = [ np.nan_to_num(st[np.argwhere(np.array(st)>1500)[0][0]:], nan=0., posinf=0., neginf=0.).tolist() for st in scan_spiketrains]
    
    start_time = 1500
    exp_tstart = start_time
    stop_time = max([max(st) if len(st) else 0 for st in scan_spiketrains])
    
    print("... producing spike rasterplot per session")
    fig = plt.figure()
    for row,train in enumerate(scan_spiketrains):
        plt.scatter( train, [row]*len(train), marker='o', edgecolors='none', s=1, c='k' )
    plt.ylabel("cell IDs")
    plt.xlabel("time (s)")
    fig.savefig(exp_path+'/results/rasterplot_twoPhotonData_scan%s.png'%scan_id, transparent=True, dpi=800)
    plt.close()
    # fig.clear()
    # fig.clf()
    
    ophys_cell_ids = list(range(len(scan_spiketrains)))
    ophys_cell_indexes = ophys_cell_ids # alias
    extra_toBremoved = sessions_evt_toBremoved[scan_id]
    
    # --------------------------------------------------------------------------
    %run "dynamical_analysis.ipynb"

    global_events[scan_id].extend(events)
    global_events_vector[scan_id].extend(events_vectors)
    global_events_sec.append(events_sec)
    global_events_duration.extend(events_durations_f)
    global_cluster_number.append(nclusters)
    global_cluster_selfsimilarity.extend(reproducibility_list)

    # --------------------------------------------------------------------------
    # Functional structure analysis
    spiketrains = scan_spiketrains
    %run "functional_analysis.ipynb"
    all_pagerank_cores.extend(pagerank_cores)
    all_pagerank_others.extend(pagerank_others)
    
# totals
print('... Total PageRank centrality')
fig, ax = plt.subplots()
xs = np.random.normal(1, 0.04, len(all_pagerank_cores))
plt.scatter(xs, all_pagerank_cores, alpha=0.3, c='forestgreen')
vp = ax.violinplot([all_pagerank_cores], widths=0.15, showextrema=False, showmedians=True)
for pc in vp['bodies']:
    pc.set_edgecolor('black')
for pc,cb in zip(vp['bodies'],['#228B224d','#D3D3D34d']):
    pc.set_facecolor(cb)
vp['cmedians'].set_color('orange')
vp['cmedians'].set_linewidth(2.)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylabel('PageRank')
plt.xticks([1, 2], ["core\n(n={:d})".format(len(all_pagerank_cores))])
fig.savefig(exp_path+"/results/global_cores_others_allpagerank.svg", transparent=True)
plt.show()



1/29
... loaded 279 spiketrains
... producing spike rasterplot per session
    population firing: 34.66±37.07 sp/frame
    cells firing rate: 0.12±0.33 sp/s
... generating surrogates to establish population event threshold
... loaded surrogates
    event size threshold (mean): 43.6550781796016
    find peaks
    find minima
    find population events
    signatures of population events
    number of events: 60
    number of events per sec: 0.037025609379821044
    events duration: 1.000±0.518
    events size: 200.500±45.230
    Similarity of events matrix
    clustering - linkage
    surrogate events signatures for clustering threshold
   ... loaded surrogates
    cluster reproducibility threshold: 0.0
    cluster size threshold: 2
    Total number of clusters: 28
    # clusters (after removing those below reproducibility threshold): 19
    event color assignment
    unique color assignments
    clustered similarity map
    finding cluster cores
    removing cores firing unspecifical